# OWL tutorial

First install owlready2 if you don't already have it, and have a quick look at the [documentation](https://owlready2.readthedocs.io/en/v0.36/onto.html#).

In [1]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
#import sys
#!{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import DC, FOAF

from owlready2 import *

Let's start loading some data from a .CSV file. We are going to create an ontology that describes the data inside.
We already did part of this using the semantics of RDF(S), now we'll use the semantics of [OWL](https://www.w3.org/TR/2012/REC-owl2-primer-20121211/) through owlready2. 

Remember that an ontology is often an application ontology, meaning that it is built with a specific task in mind. 
We could model _everything_ within a certain domain in the most ontologically correct way possible, _or_ **we could model the domain in accordance with the application's task.** 


**Your task and domain:** You are a broadcaster that has just digitised its radio archives into a digital music archive (DMA), and aims to play more interesting tracks by discovering their 'hidden treasures', by making unexpected and potentially interesting relations between tracks visible to the users (which are journalists and program makers).


**Exercise 1** 

1. look at the .csv files in the folder /data/musicoset_metadata/ and load them into pandas dataframes (use display.max_columns to show all columns). 
2. initialise an empty ontology using owlready2
3. using owlready2, create a hierarchy of classes and subclasses that describe the entities in your dataframes
4. using owrleady2, create properties and subproperties their properties, and how the classes relate to one another (using domain and range). If it helps: draw out your ontology in https://app.diagrams.net/
    - create: object properties, data properties, functional properties
5. using owlready2, add class restrictions
6. create invididuals of your classes, and provide them with attributes using your properties! 
7. write simple queries to retrieve your individuals following: https://owlready2.readthedocs.io/en/v0.36/onto.html#simple-queries. What kind of things would journalists and program makers like to retrieve? 
6. save your asserted owl file

In [2]:
# 1. load each csv in a different df
pd.options.display.max_rows
folder = "./data/musicoset_metadata/"

albums = pd.read_csv(folder + "albums.csv", sep='\t')
artists = pd.read_csv(folder + "artists.csv", sep='\t')
releases = pd.read_csv(folder + "releases.csv", sep='\t')
songs = pd.read_csv(folder + "songs.csv", sep='\t')
tracks = pd.read_csv(folder + "tracks.csv", sep='\t')

In [3]:
# 2. initialize empty ontology
ontology = get_ontology("http://example.org/myontology.owl")

### Dataframes

In [4]:
albums

,album_id,name,billboard,artists,popularity,total_tracks,album_type,image_url
0,5n1GSzC1Reao29ScnpLYqp,Dying To Live,Dying To Live,{'46SHBwWsqBkxI7EeeBEQG7': 'Kodak Black'},83,16,album,https://i.scdn.co/image/db2133234d458f432ca207...
1,6UYZEYjpN1DYRW0kqFy9ZE,Championships,Championships,{'20sxb77xiYeusSH8cVdatc': 'Meek Mill'},85,19,album,https://i.scdn.co/image/77eb7c17cafe5503c58661...
2,7uVimUILdzSZG4KKKWToq0,Christmas (Deluxe Special Edition),Christmas,{'1GxkXlMwML1oSg5eLPiAz3': 'Michael Bublé'},60,20,album,https://i.scdn.co/image/2d6ee8d4fb5a45abf35cd3...
3,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,Spider-Man: Into The Spider-Verse,{'0LyfQWJT6nXafLPZqxe9Of': 'Various Artists'},92,13,compilation,https://i.scdn.co/image/3aa37254a41cf96e815725...
4,41GuZcammIkupMPKH2OJ6I,ASTROWORLD,ASTROWORLD,{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},91,17,album,https://i.scdn.co/image/cdca7dc20c778ada42fb18...
...,...,...,...,...,...,...,...,...
26514,6AKvqcMFVMS7qz1QCOYvdd,"Mozart: Symphonie Nr. 41 C-Dur KV 551, Schuber...",Mozart: Requiem Mass,{'0K23lQ2hSQAlxSEeZ05bjI': 'Boston Symphony Or...,8,6,album,https://i.scdn.co/image/d186bb3e069972b56fcf35...
26515,0Gt9RgFYh2jMvv5ix5lCA1,"Um, Um, Um, Um, Um, Um","Um, Um, Um, Um, Um, Um/The Best Of Major Lance",{'7onp6ew3LGoQImTt1I78gt': 'Major Lance'},0,1,single,https://i.scdn.co/image/caab7f5876f71fdc54b52f...
26516,5Cw32z9ICOrrbS8Q5JcpDv,Portrait,Woody Herman: 1964,{'2KSxJY1WxGGVYSmoM0N54P': 'Woody Herman'},37,15,album,https://i.scdn.co/image/ab3877b713884d2369aa18...
26517,0B9f1PqUNLUEkNHxJPrtbR,There's A Meetin' Here Tonite,There's A Meetin' Here Tonite,"{'4loTsy7881oe7pr52sewdg': 'Eddie Brown', '2H2...",7,12,album,https://i.scdn.co/image/0bb2d05acecd9ac163de0e...


In [5]:
albums.album_type.unique()

array(['album', 'compilation', 'single'], dtype=object)

In [6]:
artists

,artist_id,name,followers,popularity,artist_type,main_genre,genres,image_url
0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,34554242.0,96,singer,dance pop,"['dance pop', 'pop', 'post-teen pop']",https://i.scdn.co/image/b1dfbe843b0b9f54ab2e58...
1,26VFTg2z8YR0cCuwLzESi2,Halsey,7368242.0,90,singer,dance pop,"['dance pop', 'electropop', 'etherpop', 'indie...",https://i.scdn.co/image/22a5f3d8c42bc7cb55215e...
2,0Y5tJX1MQlPlqiwlOH1tJY,Travis Scott,6313709.0,94,rapper,pop,"['pop', 'pop rap', 'rap']",https://i.scdn.co/image/dc5eba5e032c2e5bc4d42c...
3,246dkjvS1zLTtiykXe5h60,Post Malone,16737002.0,96,rapper,dfw rap,"['dfw rap', 'pop', 'rap']",https://i.scdn.co/image/f9d8b742b66609f12da023...
4,1zNqQNIdeOUZHb8zbZRFMX,Swae Lee,483032.0,89,singer,trap music,['trap music'],https://i.scdn.co/image/a177469870b41f7e17e3b5...
...,...,...,...,...,...,...,...,...
11513,7vyRisgvM6Wm0Pnp0qXx6m,Sweeney Todd,634.0,19,-,classic canadian rock,['classic canadian rock'],https://i.scdn.co/image/47166b7afefd590ce85086...
11514,2Uh4UmiQhrrElbrvJVH0dT,Brooklyn Dreams,318.0,32,band,-,[],https://i.scdn.co/image/bae9c82929c50d2dadb1ec...
11515,1VGFS4UGLOAxlMGqzcqHG1,PMD,1405.0,32,-,-,[],https://i.scdn.co/image/926172541a4dff5dca6bce...
11516,3Se8xpgCBmfXVnZqRSRRH9,The Tribute Co.,274.0,21,-,-,[],https://i.scdn.co/image/d4e9b2299fc5cbe4332251...


In [7]:
artists.artist_type.unique()

array(['singer', 'rapper', 'DJ', 'band', '-', 'duo', "'band'"],
      dtype=object)

In [8]:
artists.main_genre.unique()[:10]
print(f"{len(artists.main_genre.unique())} genres")

992 genres


In [9]:
releases

,artist_id,album_id,release_date,release_date_precision
0,46SHBwWsqBkxI7EeeBEQG7,5n1GSzC1Reao29ScnpLYqp,2018-12-14,day
1,20sxb77xiYeusSH8cVdatc,6UYZEYjpN1DYRW0kqFy9ZE,2018-11-30,day
2,1GxkXlMwML1oSg5eLPiAz3,7uVimUILdzSZG4KKKWToq0,2012-11-09,day
3,0LyfQWJT6nXafLPZqxe9Of,35s58BRTGAEWztPo9WqCIs,2018-12-14,day
4,0Y5tJX1MQlPlqiwlOH1tJY,41GuZcammIkupMPKH2OJ6I,2018-08-03,day
...,...,...,...,...
26517,6XVxGOzn7c2rLi1GpXxm5v,6AKvqcMFVMS7qz1QCOYvdd,2002-01-01,day
26518,7onp6ew3LGoQImTt1I78gt,0Gt9RgFYh2jMvv5ix5lCA1,2014-03-10,day
26519,2KSxJY1WxGGVYSmoM0N54P,5Cw32z9ICOrrbS8Q5JcpDv,2014-04-18,day
26520,4loTsy7881oe7pr52sewdg,0B9f1PqUNLUEkNHxJPrtbR,2011-09-30,day


In [10]:
songs

,song_id,song_name,billboard,artists,popularity,explicit,song_type
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next","('Thank U, Next', 'Ariana Grande')",{'66CXWjxzNUsdJxJ2JdwvnR': 'Ariana Grande'},86,True,Solo
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,"('Without Me', 'Halsey')",{'26VFTg2z8YR0cCuwLzESi2': 'Halsey'},87,True,Solo
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,"('Sicko Mode', 'Travis Scott')",{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},85,True,Solo
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,('Sunflower (Spider-Man: Into The Spider-Verse...,"{'246dkjvS1zLTtiykXe5h60': 'Post Malone', '1zN...",92,False,Collaboration
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,"('High Hopes', 'Panic! At The Disco')",{'20JZFwl6HVl6yg8a4H3ZqK': 'Panic! At The Disco'},86,False,Solo
...,...,...,...,...,...,...,...
20400,4NnhLA66RRLXxKbiiscU9R,How Can I Be Sure - Single Version,"('How Can I Be Sure', 'David Cassidy')",{'5X3TuTi9OIsJXMGxPwTKM2': 'The Young Rascals'},36,False,Solo
20401,2jHfXdCLibrI1J56LnUAZv,To Show I Love You - Mono,"('To Show I Love You', 'Peter And Gordon')",{'6lHC2EQMEMZiEmSfFloarn': 'Peter And Gordon'},1,False,Solo
20402,6zqsyB7uIvWrL1iCJzpNrs,You Better Run - Single Version,"('You Better Run', 'Pat Benatar')",{'5X3TuTi9OIsJXMGxPwTKM2': 'The Young Rascals'},20,False,Solo
20403,5mz9pQZZXNpAw9CdQ7Bk8q,Don't Pity Me - Mono,"('Don't Pity Me', 'Peter And Gordon')",{'6lHC2EQMEMZiEmSfFloarn': 'Peter And Gordon'},7,False,Solo


In [11]:
songs.song_type.unique()

array(['Solo', 'Collaboration'], dtype=object)

In [12]:
tracks

,song_id,album_id,track_number,release_date,release_date_precision
0,3e9HZxeyfWwjeyPAMmWSSQ,2fYhqwDWXjbpjaIJPEfKFw,11,2019-02-08,day
1,5p7ujcrUXASCNwRaWNHR1C,0zzrCTzvL4ZmR42xF46Afm,1,2018-10-04,day
2,2xLMifQCjDGFmkHkpNLD9h,41GuZcammIkupMPKH2OJ6I,3,2018-08-03,day
3,3KkXRkHbMCARz0aVfEt68P,35s58BRTGAEWztPo9WqCIs,2,2018-12-14,day
4,1rqqCSm0Qe4I9rUvWncaom,6ApYSpXF8GxZAgBTHDzYge,4,2018-06-22,day
...,...,...,...,...,...
20400,0yZsBLZVU2HTMlMqYvWevJ,5UoKanu6wcBQhwfPSDHYVw,4,2011-04-01,day
20401,30342v7I9sUoNC0Djnu1mW,1bZHbHtUvjGqUOKNla4lo0,25,2000-01-01,day
20402,5rkgbGIOTBGlKTAawWb06X,3GmCXW10kLxmZrEY0JpRlw,9,1988-03-07,day
20403,2MVPsa4S4r7Ii5lMh85E03,0UiVuqXa69zXqql0GoXH2h,22,2013-01-31,day


### Cont.

In [13]:
# 3. create classes - each class is subclass of thing
class Artist(Thing):
    namespace = ontology

class Location(Thing):
    namespace = ontology

class Album(Thing):
    namespace = ontology

class Genre(Thing):
    namespace = ontology

class Song(Thing):
    namespace = ontology

In [14]:
class Compilation(Album):
    pass

class Single(Album):
    pass

class Solo(Song):
    pass

class Collaboration(Song):
    pass

In [15]:
print(list(ontology.classes()))

[myontology.Artist, myontology.Location, myontology.Album, myontology.Genre, myontology.Song, myontology.Compilation, myontology.Single, myontology.Solo, myontology.Collaboration]


In [16]:
print(Single.ancestors())

{owl.Thing, myontology.Single, myontology.Album}


In [17]:
# 4. create properties and subproperties

# object properties

class songHasGenre(Song >> Genre):
    namespace = ontology

class performedBy(Song >> Artist):
    namespace = ontology

class contains(Album >> Song):
    namespace = ontology

class locatedIn(Artist >> Location):
    namespace = ontology

# restrictions

class includedIn(Song >> Album):
    inverse_property = contains 
    namespace = ontology

class locatedIn(ObjectProperty, TransitiveProperty):
    namespace = ontology
    domain = [Location]
    range = [Location]

# data properties
# functional properties

class artistName(DataProperty, FunctionalProperty):
    domain = [Artist]
    range = [str]
    namespace = ontology

class numberOfFollowers(DataProperty, FunctionalProperty):
    domain = [Artist]
    range = [int]
    namespace = ontology

class releaseDate(DataProperty, FunctionalProperty):
    domain = [Song]
    range = [str]
    namespace = ontology

In [18]:
list(ontology.properties())

[myontology.artistName,
 myontology.numberOfFollowers,
 myontology.releaseDate,
 myontology.songHasGenre,
 myontology.performedBy,
 myontology.contains,
 myontology.locatedIn,
 myontology.includedIn]

In [19]:
# 5. add class restrictions

Song.is_a.append(songHasGenre.min(1, Genre))
Song.is_a.append(performedBy.min(1, Artist))
Album.is_a.append(contains.min(1, Song))

Artist.is_a.append(artistName.exactly(1))
Song.is_a.append(releaseDate.exactly(1))

In [20]:
class Collaboration(Song):
    equivalent_to = [Song & performedBy.min(2)]
    
class Solo(Song):
    equivalent_to = [Song & performedBy.exactly(1)]

In [21]:
AllDisjoint([Artist, Song])
AllDisjoint([Genre, Song])
AllDisjoint([Genre, Album])
AllDisjoint([Genre, Artist])

AllDisjoint([myontology.Genre, myontology.Artist])

In [22]:
# 6. create instances of the classes

halsey = Artist("Halsey", artistName="Ashley Nicolette Frangipane", numberOfFollowers=31700000)
yungblud = Artist("Yungblud", artistName="Dominic Richard Harrison", numberOfFollowers=3800000)

graveyard = Solo("Graveyard", releaseDate="13-09-2019", performedBy=[halsey])
electropop = Genre("Electropop")
graveyard.songHasGenre = [electropop]
manic = Album("Manic", contains=[graveyard])

eleven_minutes = Collaboration("11Minutes", releaseDate="13-02-2019", performedBy=[halsey, yungblud])
rap_rock = Genre("RapRock")
eleven_minutes.songHasGenre = [rap_rock]
eleven_single = Single("11MinutesSingle", contains=[eleven_minutes])

Genre.is_a.append(OneOf([electropop, rap_rock]))
AllDifferent([electropop, rap_rock])

AllDisjoint([myontology.Electropop, myontology.RapRock])

In [23]:
usa = Location('USA')
ca = Location("CA", locatedIn = [usa])
la = Location("LA", locatedIn = [ca])

halsey.locatedIn = [la]

In [24]:
# 7. write simple queries HINT (onto.search, ..)

for d in ontology.disjoints():
    print(d.entities)

[myontology.Artist, myontology.Song]
[myontology.Genre, myontology.Song]
[myontology.Genre, myontology.Album]
[myontology.Genre, myontology.Artist]
[myontology.Electropop, myontology.RapRock]


In [25]:
print(f"Halsey's IRI : {halsey.iri}")
print(f"Halsey's name : {halsey.artistName}")
print("\n")  
print(f"Songs that are included in some albums : {ontology.search(includedIn = '*')}")
print("\n")  
print(f"Songs that are included in the Manic album : {ontology.search(includedIn = [manic])}")
print(f"Release date of the Graveyard song : {graveyard.releaseDate}")
print(f"Genre of the Graveyard song : {graveyard.songHasGenre}")
print(f"Graveyard is performed by : {graveyard.performedBy}")
print("\n")  
print(f"Songs that are included in the 11 Minutes album : {ontology.search(includedIn = [eleven_single])}")
print(f"Release date of the 11 Minutes song : {eleven_minutes.releaseDate}")
print(f"Genre of the 11 Minutes song : {eleven_minutes.songHasGenre}")
print(f"11 Minutes is performed by : {eleven_minutes.performedBy}")
print("\n")  
print(f"Search for a IRI containing 'Minutes' : {ontology.search(iri = '*Minutes*')}")

Halsey's IRI : http://example.org/myontology.owl#Halsey
Halsey's name : Ashley Nicolette Frangipane


Songs that are included in some albums : [myontology.Graveyard, myontology.11Minutes]


Songs that are included in the Manic album : [myontology.Graveyard]
Release date of the Graveyard song : 13-09-2019
Genre of the Graveyard song : [myontology.Electropop]
Graveyard is performed by : [myontology.Halsey]


Songs that are included in the 11 Minutes album : [myontology.11Minutes]
Release date of the 11 Minutes song : 13-02-2019
Genre of the 11 Minutes song : [myontology.RapRock]
11 Minutes is performed by : [myontology.Halsey, myontology.Yungblud]


Search for a IRI containing 'Minutes' : [myontology.11Minutes, myontology.11MinutesSingle]


In [26]:
# 8. save
ontology.save(file = "./data/my_music_ontology_asserted.owl", format = "rdfxml")

## OWL reasoning 

Let's look at how reasoning works.

Owlready automatically gets the results of the reasoning from HermiT (a type of reasoner) and reclassifies Individuals and Classes. 

**Exercise 2**
1. think about which things are inferred from your OWL semantics. Query/look at your graph: do you see what you expected?
2. looking at the following tutorial [owlready2-reasoning](https://owlready2.readthedocs.io/en/latest/reasoning.html), which things have not yet been inferred? Run the owlready2 reasoner to:
    - infer these new triples
    - check your ontology and statements (individuals + attributes) for consistency
3. save your asserted + inferred triples to a new file 

In [27]:
# 1.

print(f"Songs that are included in the Manic album : {ontology.search(includedIn = [manic])}")
print(f"Songs that are included in the 11 Minutes album : {ontology.search(includedIn = [eleven_single])}")
print("\nAlbums:")
for individual in Album.instances():
    print(individual)
print("\nSongs:")
for individual in Song.instances():
    print(individual)

Songs that are included in the Manic album : [myontology.Graveyard]
Songs that are included in the 11 Minutes album : [myontology.11Minutes]

Albums:
myontology.Manic
myontology.11MinutesSingle

Songs:
myontology.Graveyard
myontology.11Minutes


In [28]:
halsey.locatedIn

[myontology.LA]

In [29]:
# 2.

with ontology: 
    sync_reasoner(infer_property_values=True)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/arina/.pyenv/versions/3.10.13/envs/krwvenv/lib/python3.10/site-packages/owlready2/hermit:/Users/arina/.pyenv/versions/3.10.13/envs/krwvenv/lib/python3.10/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/6w/dsgqffvn3sb_69bf9c_sd0000000gn/T/tmpq45hyt9w -Y


* Owlready * Adding relation myontology.Halsey locatedIn myontology.USA
* Owlready * Adding relation myontology.Halsey locatedIn myontology.CA
* Owlready * Adding relation myontology.LA locatedIn myontology.USA


* Owlready2 * HermiT took 0.5083370208740234 seconds
* Owlready * Reparenting myontology.Halsey: {myontology.Artist} => {myontology.Location, myontology.Artist}
* Owlready * Reparenting myontology.CA: {myontology.Location} => {myontology.Location, myontology.Artist}
* Owlready * Reparenting myontology.LA: {myontology.Location} => {myontology.Location, myontology.Artist}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [30]:
halsey.locatedIn

[myontology.LA, myontology.USA, myontology.CA]

In [31]:
list(default_world.inconsistent_classes())

[]

In [32]:
if Nothing in Thing.equivalent_to:
    print("Thing is inconsistent!")

In [33]:
# 3.

ontology.save(file = "./data/my_music_ontology_inferred.owl", format = "rdfxml")

#### Querying inferred triples

**Exercise 3**
Query your inferred triples: 

- *.get_parents_of(entity)* accepts any entity (Class, property or individual), and returns the superclasses (for a class), the superproperties (for a property), or the classes (for an individual). 

- *.get_instances_of(Class)* returns the individuals that are asserted as belonging to the given Class in the ontology. (NB for obtaining all instances, independently of the ontology they are asserted in, use Class.instances()).

- *.get_children_of(entity)* returns the subclasses (or subproperties) that are asserted for the given Class or property in the ontology. (NB for obtaining all children, independently of the ontology they are asserted in, use entity.subclasses()).

In [37]:
# 1. 

print(ontology.get_parents_of(graveyard))
print(ontology.get_parents_of(Solo))
print(ontology.get_parents_of(la))

[myontology.Solo]
[myontology.Song]
[myontology.Location, myontology.Artist]


In [35]:
# 2.

print(ontology.get_instances_of(Genre))
print(ontology.get_instances_of(Album))
print(ontology.get_instances_of(Single))
print(Album.instances())
print(Song.instances())

[myontology.Electropop, myontology.RapRock]
[myontology.Manic]
[myontology.11MinutesSingle]
[myontology.Manic, myontology.11MinutesSingle]
[myontology.Graveyard, myontology.11Minutes]


In [36]:
# 3.

print(ontology.get_children_of(Song))
print(ontology.get_children_of(Album))
print(list(Album.subclasses()))

[myontology.Solo, myontology.Collaboration]
[myontology.Compilation, myontology.Single]
[myontology.Compilation, myontology.Single]
